# Bancor Subgraph

https://thegraph.com/explorer/subgraph/blocklytics/bancor-v2

In [108]:
from graphqlclient import GraphQLClient
import json
import datetime as dt

In [7]:
ENDPOINT = "https://api.thegraph.com/subgraphs/name/blocklytics/bancor-v2"
client = GraphQLClient(ENDPOINT)

## Pool Volume

Pool volumes are given in terms of each token in the pool and for the buy, sell and overall volume.

To get the lifetime volume for a given pool (converter):

In [141]:
def get_volumes(converter):
    QUERY = """
        {{
          converter(id:"{0}") {{
            id
            activated
            volumes {{
              token {{ symbol }}
              sellVolume
              buyVolume
              totalVolume
            }}
            createdAtTimestamp
          }}
        }}""".format(converter)

    return json.loads(client.execute(QUERY))

result = get_volumes("0xd5aa38c6df168741839f9e1e9c51e6d24b986970")
result

{'data': {'converter': {'activated': True,
   'createdAtTimestamp': '1599042695',
   'id': '0xd5aa38c6df168741839f9e1e9c51e6d24b986970',
   'volumes': [{'buyVolume': '25774.886191898205036102',
     'sellVolume': '35143.005382287974667567',
     'token': {'symbol': 'BNT'},
     'totalVolume': '60917.891574186179703669'},
    {'buyVolume': '125362.519681479419761064',
     'sellVolume': '94858.229518880359971895',
     'token': {'symbol': 'REN'},
     'totalVolume': '220220.749200359779732959'}]}}}

In [142]:
def display_volumes(data):
    converter = data['id']
    converter_creation_timestamp = int(data['createdAtTimestamp'])
    created_at = datetime.fromtimestamp(converter_creation_timestamp)
    print("Volumes for {0}\n".format(converter))

    for row in data['volumes']:
        buy_volume = row['buyVolume']
        sell_volume = row['sellVolume']
        total_volume = row['totalVolume']
        symbol = row['token']['symbol']

        print("====={0} Volume=====".format(symbol))
        print("Buy:   {0} {1}".format(buy_volume, symbol))
        print("Sell:  {0} {1}".format(sell_volume, symbol))
        print("Total: {0} {1}".format(total_volume, symbol))
        print("Since {0}\n".format(created_at))

display_volumes(result['data']['converter'])

Volumes for 0xd5aa38c6df168741839f9e1e9c51e6d24b986970

=====BNT Volume=====
Buy:   25774.886191898205036102 BNT
Sell:  35143.005382287974667567 BNT
Total: 60917.891574186179703669 BNT
Since 2020-09-02 10:31:35

=====REN Volume=====
Buy:   125362.519681479419761064 REN
Sell:  94858.229518880359971895 REN
Total: 220220.749200359779732959 REN
Since 2020-09-02 10:31:35



If we want to go further back for this pair, we can find the previous converter using `upgradedFrom`, which is another converter address...

In [143]:
QUERY = """
    {{
      converter(id:"{0}") {{
        upgradedFrom {{
          id
        }}
      }}
    }}""".format(EXCHANGE)

result = json.loads(client.execute(QUERY))
previous_converter = result['data']['converter']['upgradedFrom']['id']
previous_converter

'0x2eb3eda19e9fb1f821dfb379c485062ea90df779'

In [145]:
result = get_volumes(previous_converter)
display_volumes(result['data']['converter'])

Volumes for 0x2eb3eda19e9fb1f821dfb379c485062ea90df779

=====BNT Volume=====
Buy:   73822.762794205374555346 BNT
Sell:  26477.39717237874731916 BNT
Total: 100300.159966584121874506 BNT
Since 2020-08-26 20:17:09

=====REN Volume=====
Buy:   109597.522607717892782943 REN
Sell:  291952.806734791927935279 REN
Total: 401550.329342509820718222 REN
Since 2020-08-26 20:17:09



## Rolling Volume

Normally, we don't care about lifetime volume and only want to know volume for the last `1d`, `7d`, `30d`. In this case, we can use time-travel queries.

We will specify a block number to compare `before` volumes with `after` volumes. For example:

In [146]:
block_number = 10760000

def get_exchange_snapshot_volume(exchange, block_number):
    QUERY = """
        {{
          before:converter(
            id:"{0}"
            block:{{
              number:{1}
            }}
          ) {{
            id
            volumes {{
              token {{ symbol }}
              totalVolume
            }}
          }}
          after:converter(
            id:"{0}"
          ) {{
            id
            volumes {{
              token {{ symbol }}
              totalVolume
            }}
          }}
        }}""".format(exchange, block_number)

    return json.loads(client.execute(QUERY))

result = get_exchange_snapshot_volume(previous_converter, block_number)
result

{'data': {'after': {'id': '0x2eb3eda19e9fb1f821dfb379c485062ea90df779',
   'volumes': [{'token': {'symbol': 'BNT'},
     'totalVolume': '100300.159966584121874506'},
    {'token': {'symbol': 'REN'}, 'totalVolume': '401550.329342509820718222'}]},
  'before': {'id': '0x2eb3eda19e9fb1f821dfb379c485062ea90df779',
   'volumes': [{'token': {'symbol': 'BNT'},
     'totalVolume': '16030.182917770475928784'},
    {'token': {'symbol': 'REN'}, 'totalVolume': '70571.627746303023652931'}]}}}

In [147]:
def display_volume_diffs(data):
    before_volumes = data['before']['volumes']
    after_volumes = data['after']['volumes']

    for before_row in before_volumes:
        for after_row in after_volumes:
            if before_row['token']['id'] == after_row['token']['id']:
                symbol = before_row['token']['symbol']
                before_volume = float(before_row['totalVolume'])
                after_volume = float(after_row['totalVolume'])
                diff_volume = after_volume - before_volume

                print("===={0} Volume=====".format(symbol))
                print("Before: {0} {1}".format(before_volume, symbol))
                print("After:  {0} {1}".format(after_volume, symbol))
                print("Diff: {0} {1}\n".format(diff_volume, symbol))

display_volume_diffs(result['data'])

====BNT Volume=====
Before: 16030.182917770477 BNT
After:  100300.15996658412 BNT
Diff: 84269.97704881364 BNT

====REN Volume=====
Before: 70571.62774630303 REN
After:  401550.3293425098 REN
Diff: 330978.70159620675 REN



But... blocks are not a human-friendly measurement of time! Use helper functions to change from time to blocks:

In [148]:
def timestamp_for_days_ago(days):
    return int((dt.datetime.now() - dt.timedelta(days=days)).timestamp())

def block_for_timestamp(timestamp):
    ethereum_blocks_subgraph = "https://api.thegraph.com/subgraphs/name/blocklytics/ethereum-blocks"
    ethereum_blocks_client = GraphQLClient(ethereum_blocks_subgraph)
    ethereum_blocks_query = """
        {{
            blocks(
                first: 1, 
                orderBy: number, 
                orderDirection: asc, 
                where: {{ timestamp_gte: "{0}", timestamp_lt: "{1}"}}
            ) {{
                number
            }}
        }}""".format(timestamp, timestamp+600)

    result = json.loads(ethereum_blocks_client.execute(ethereum_blocks_query))
    return result['data']['blocks'][0]['number']

yesterday = timestamp_for_days_ago(1)
block_for_timestamp(yesterday)

'10788607'

In [149]:
days_ago = 7
timestamp = timestamp_for_days_ago(days_ago)
block = block_for_timestamp(timestamp)
result = get_exchange_snapshot_volume(previous_converter, block)

print(exchange)
print("{0}d Volume...\n".format(days_ago))
display_volume_diffs(result['data'])

0x2eb3eda19e9fb1f821dfb379c485062ea90df779
7d Volume...

====BNT Volume=====
Before: 4881.806032585283 BNT
After:  100300.15996658412 BNT
Diff: 95418.35393399883 BNT

====REN Volume=====
Before: 20037.94612876574 REN
After:  401550.3293425098 REN
Diff: 381512.38321374403 REN



# Network Volume

The same concepts can be applied to network volume:

In [164]:
network_query = """
    {{
        volumeStats {{
          token {{ 
              id
              symbol 
          }}
          totalVolume
        }}
    }}""".format()

result = json.loads(client.execute(network_query))
result['data']['volumeStats']

# Note: volume is not aggregated by token.
# There are duplicate results if different converts have the same tokens.

[{'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'},
  'totalVolume': '861236.899951375840563878'},
 {'token': {'id': '0x960b236a07cf122663c4303350609a66a7b288c0',
   'symbol': 'ANT'},
  'totalVolume': '325191.874366941064876337'},
 {'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'},
  'totalVolume': '380764.794712048490659657'},
 {'token': {'id': '0x514910771af9ca656af840dff83e8264ecf986ca',
   'symbol': 'LINK'},
  'totalVolume': '40774.748157896463985499'},
 {'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'},
  'totalVolume': '279347.470024054425230272'},
 {'token': {'id': '0xb4efd85c19999d84251304bda99e90b92300bd93',
   'symbol': 'RPL'},
  'totalVolume': '170387.376760814664278124'},
 {'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'},
  'totalVolume': '43.230824947580821095'},
 {'token': {'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
   'symbol': 'US

We could filter the volume stats just for BNT `0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c` as follows:

In [165]:
bnt_token = "0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c"
network_query = """
    {{
        volumeStats(where:{{
            token: "{0}"
        }}) {{
          token {{ 
              id
              symbol 
          }}
          totalVolume
        }}
    }}""".format(bnt_token)

result = json.loads(client.execute(network_query))
result['data']['volumeStats']

[{'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'},
  'totalVolume': '861236.899951375840563878'},
 {'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'},
  'totalVolume': '380764.794712048490659657'},
 {'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'},
  'totalVolume': '279347.470024054425230272'},
 {'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'},
  'totalVolume': '43.230824947580821095'},
 {'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'},
  'totalVolume': '94.563638630092571364'},
 {'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'},
  'totalVolume': '1990151.557250318078804662'},
 {'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'},
  'totalVolume': '2577506.498352379233587353'},
 {'token': {'id': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c',
   'symbol': 'BNT'

In [166]:
bnt_volume = 0
for row in result['data']['volumeStats']:
    bnt_volume += float(row['totalVolume'])

bnt_volume

16562055.946446287

And combining concepts to only look at 7d volume:

In [168]:
days_ago = 7
timestamp = timestamp_for_days_ago(days_ago)
block = block_for_timestamp(timestamp)

bnt_token = "0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c"
network_query = """
    {{
        volumeStats(
        block:{{number: {0}}}
        where:{{
            token: "{1}"
        }}) {{
          token {{ 
              id
              symbol 
          }}
          totalVolume
        }}
    }}""".format(block, bnt_token)

result = json.loads(client.execute(network_query))

bnt_volume_before = 0
for row in result['data']['volumeStats']:
    bnt_volume_before += float(row['totalVolume'])

bnt_volume - bnt_volume_before

1128654.5112037323

# Pool Token Returns

APR data: 1d, 7d, 30d

See query https://thegraph.com/explorer/subgraph/blocklytics/bancor-v2?query=Share%20Value%20over%20Time

# Trading Fees Collected

Fees: 1d, 7d, 30d

It's possible to see trading fees on a per-swap basis. See query below:

In [ ]:
query = `
{
  swaps(first: 5, orderBy: createdAtBlockNumber, orderDirection: desc) {
    toToken {
      symbol
      decimals
    }
    conversionFee
    converter {
      id
    }
  }
}
`